In [ ]:
import os
import sys
import pathlib
import subprocess

import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

####################################################################
### NOTE: CHANGE PATHS FOR NRSS, PYHYPER, AND CURRENT REPOSITORY ###
####################################################################

sys.path.append('/home/devon/Documents/Github/NRSS/')
sys.path.append('/home/devon/Documents/Github/PyHyperScattering/src/')
sys.path.append('/home/devon/Documents/DopantModeling/')

# NOTE: C
import experiments.ex.params as p


from src.Common.files import make_output_dir
from src.Morphology.Fibril import FibrilGenerator
from src.Morphology.Fibril import FibrilPostProcessor
from src.Simulation import cyrsoxs
from src.Visualization import plot, animation

%load_ext autoreload
%autoreload 2

In [ ]:
##########################################
### DEFINE EXPERIMENT SWEEP PARAMETERS ###
##########################################

exp1 = {
    "45": 45,
    # "60": 60,
    # "75": 75,
    # "90": 90
}

In [ ]:
# SINGLE VARIABLE SWEEP
for exp1_name, exp1_val in exp1.items():

    # # Create experiment save directory
    dir_name = f'{exp1_name}_nm'
    save_dir = make_output_dir(p.base_path, dir_name, delete_dir_if_exists=False)
    fig_dir = make_output_dir(save_dir, 'figures', delete_dir_if_exists=False)

    # Adjust input parameters based on experiment values
    p.c2c_dist_nm = exp1_val

    # Build morphology 
    fibgen = FibrilGenerator.generate_morphology(p)
    data = FibrilPostProcessor.process_morphology(fibgen, p)
    cyrsoxs.create_hdf5(data, p)
    figs = plot.plot_checkH5(p.DEFAULT_MORPH_FILE, z_slice=int(p.z_dim_nm/p.pitch_nm/2), save_dir=fig_dir)

    # Create cyrsoxs inputs
    cyrsoxs.create_inputs(p)

    # Simulate scattering
    cyrsoxs.run(p, save_dir=save_dir)
    cyrsoxs.cleanup(p, delete_morph_file=False) # Change this flag 

    # Analyze/visualize results
    raw_data, integ_data = cyrsoxs.load(save_dir)
    para, perp, AR = cyrsoxs.get_para_perp_AR(integ_data, q_range=p.q_range)
    # Iq2, ISI = cyrsoxs.get_Iq2_ISI(integ_data, E_range=p.E_range)

    fig, ax = plot.plot_anisotropy_EvQ(AR, q_range=p.q_range, E_range=p.E_range, save_dir=fig_dir)
    fig, ax = plot.plot_para_perp(para, perp, q_range=p.q_range, I_range=p.I_range, E_range=p.E_range, save_dir=fig_dir)
    fig, ax = plot.plot_intensity_EvQ(para, q_range=p.q_range, E_range=p.E_range, label='Parallel', save_dir=fig_dir)
    fig, ax = plot.plot_intensity_EvQ(perp, q_range=p.q_range, E_range=p.E_range, label='Perpendicular', save_dir=fig_dir)
    # fig, ax = plot.plot_Iq2(Iq2, q_range=p.q_range, E_range=p.E_range, save_dir=fig_dir)
    # fig, ax = plot.plot_ISI(ISI, E_range=p.E_range, save_dir=fig_dir)
    # Plotting functions: checkH5, ISI, circular avg, FY NEXAFS 
